# Model Training and Evaluation
This notebook covers the process of training, tuning, and evaluating machine learning models for customer churn prediction. We will use the processed data and compare multiple algorithms.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

## 1. Load Processed Data
We will use the processed dataset generated after feature engineering and preprocessing.

In [ ]:
# Load processed data
processed = pd.read_csv('../data/processed/processed.csv')
X = processed.drop('Churn', axis=1)
y = processed['Churn']
print(f"Features shape: {X.shape}, Target shape: {y.shape}")

## 2. Train/Test Split
Split the data into training and test sets for model evaluation.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

## 3. Model Training
We will train Logistic Regression, Random Forest, and XGBoost models. Hyperparameter tuning is performed using Optuna.

In [ ]:
# Load trained models (assumes models are already trained and saved)
with open('../models/logistic_regression.pkl', 'rb') as f:
    lr_model = pickle.load(f)
with open('../models/random_forest.pkl', 'rb') as f:
    rf_model = pickle.load(f)
with open('../models/xgboost.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

## 4. Model Evaluation
Evaluate each model on the test set and compare their performance.

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    if y_proba is not None:
        auc = roc_auc_score(y_test, y_proba)
        print(f'ROC AUC: {auc:.3f}')
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        plt.plot(fpr, tpr, label=f'AUC = {auc:.3f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend()
        plt.show()

print('Logistic Regression:')
evaluate_model(lr_model, X_test, y_test)
print('Random Forest:')
evaluate_model(rf_model, X_test, y_test)
print('XGBoost:')
evaluate_model(xgb_model, X_test, y_test)

## 5. Model Comparison and Insights
Summarize the results and provide insights on which model performed best and why.

- Logistic Regression provides a good baseline and is interpretable.
- Random Forest and XGBoost often achieve higher accuracy and ROC AUC, especially with tuned hyperparameters.
- ROC curves and confusion matrices help visualize the trade-offs between models.
- Consider business context and interpretability when selecting the final model.